# 04 Embeddings | 01-BasicEmbeddings

## Azure Environment

You can create the necessary instance of Azure OpenAI and deploy an embedding model by executing one of the deployment options [mentioned here](../../create_env/src/AzureCLI/CreateEnv.azcli). To execute the sample code service specific information is needed ([Details and instructions here](../01_DemoEnvironment/01_Environment.ipynb)) 

## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. It also allows developers to specify the model, engine, and options for each request, such as temperature, frequency penalty, presence penalty, and stop sequences. 

The OpenAIClient can connect to any Azure OpenAI resource or to the non-Azure OpenAI inference endpoint, making it a versatile and powerful tool for .NET development with OpenAI.

In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("SKIT_EMBEDDING_DEPLOYMENTNAME") ?? "SKIT_EMBEDDING_DEPLOYMENTNAME not found";

string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);
Console.WriteLine($"OpenAI Client created...");


Expected output:

```
Installed Packages
    Azure.AI.OpenAI, 1.0.0-beta.12
    DotNetEnv, 2.5.0

OpenAI Client created...
```

## Step 2: Create Embeddings

The following sample creates a vector representation (embedding) from input text which should be stored in a data base and a query to perform a semantic search.
The sample uses the Azure OpenAI SDK.

To calculate the embeddings/vector you have to call the method `GetEmbeddingsAsync()` on the instance of `OpenAIClient`. The method provides multiple return values where the embedding can be found in `modelResponse.Value.Data[0]`.

In the example embeddings two embeddings/vectors are calculated using the deployed OpenAI model:
- The file `Path.Combine(assetInterimFolder, "acidocu_page_194.txt");` provides text which was extracted from a [PDF file](../../assets/maintainer_demo/azure_aci_documentation.pdf). It contains textual information from page 194 from the PDF file. 
- A hard coded string `"How to best integrate Source Control"` will be also transformed into an embedding/vector representation.

Both embeddings will be used in further samples to perform a vector based query.

In [ ]:

string documentationPage = Path.Combine(assetsFolder,"docs", "04_Embeddings" , "acidocu_page_194.txt");

string textToBeVecorized = File.ReadAllText(documentationPage);

// Vectorize input text from file
EmbeddingsOptions embeddingsOptions = new EmbeddingsOptions(embeddingDeploymentName, new List<string> { textToBeVecorized });
var modelResponse = await openAIClient.GetEmbeddingsAsync( embeddingsOptions);
float[] vectorDocumentation = modelResponse.Value.Data[0].Embedding.ToArray();

Console.WriteLine($"Vector from {documentationPage} created...");

// Vectorize question
string question = "How to best integrate Source Control?";
embeddingsOptions = new EmbeddingsOptions(embeddingDeploymentName, new List<string> { question });
modelResponse = await openAIClient.GetEmbeddingsAsync(embeddingsOptions);
float[] vectorQuery = modelResponse.Value.Data[0].Embedding.ToArray();

Console.WriteLine($"Vector from '{question}' created... Vector Length: {vectorQuery.Length}");


Expected output:

```
Vector from <path to your local>/OpenAI.StarterKit/docs/04_Embeddings/../../assets/maintainer_demo/interim_data/acidocu_page_194.txt created...
Vector from 'How to best integrate Source Control?' created... Vector Length: 1536
```

## Next step

The created embeddings/vectors can be used to perform additional processing on them like similarity search: [Demo Cosine Search](02_CosineSimilarity.ipynb).